# Preprocessing gait detection
This script preprocesses the sensor data, tabulates it into windows, and generates features. 

Execution time $\approx$ 50s per participant (amounting to 35 minutes in total).

### Modules

In [1]:
from pdathome.constants import global_constants as gc
from pdathome.preprocessing import preprocess_gait_detection

### Process data

In [ ]:
for subject in ['hbv002']:# gc.participant_ids.L_PD_IDS + gc.participant_ids.L_HC_IDS:
    preprocess_gait_detection(subject)